# Tema 4: Logging & Inference

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import Lambda
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import os
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
import seaborn as sns

### 1. Seleccionar GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

## 2. Conectar con Google Drive

Para poder guardar los modelos.

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 3. Crear el dataset y dataloader para el conjunto de Fashion MNIST.

### Función Transform

In [ ]:
transform = transforms.Compose([
    ### CÓDIGO ###

])

### Dataset

In [ ]:
full_train_dataset = datasets.FashionMNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root="./data", train=False, transform=transform, download=True)

### Dividir el conjunto de train, entre train y validación (80 y 20% respectivamente).

Para ello utilizar ***random_split*** de torch.

In [ ]:
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size

In [ ]:
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

### Crear una función collate que aplane las muestras y devuelva por separado las muestras y las etiquetas.

In [ ]:
def flatten_batch(data):

### CÓDIGO ###

    return images, labels_collate

### Crear el Dataloader

In [ ]:
train_loader = DataLoader()
val_loader = DataLoader()
test_loader = DataLoader()

## 4. Crear el modelo de MLP

Se puede utilizar la clase que hemos creado en otras sesiones de la asignatura.

In [ ]:
class MLP_pytorch(nn.Module):
    def __init__(self, input_shape=150*150, n_classes=10):
        super().__init__()

### CÓDIGO ###

    def forward(self, x):
        # Forward pass

### CÓDIGO ###

        return x

### Crear una instancia del modelo y pasarlo a GPU

In [ ]:
model = ### CÓDIGO ###

## 5. Crear las funciones para:

* model_checkpoint: que guarde el mejor modelo, utilizando el loss de validación.
* plot_progress: una función que en cada época guarde una visualización con las gráficas de loss y accuracy para entrenamiento y validación. Guardalo en PDF.
* Guardar el loss y el accuracy de cada época en un archivo csv.

In [ ]:
def model_checkpoint(val_loss,epoch,model,optimizer,checkpoint_path,best_model_path):

### CÓDIGO ###

def plot_progress(train_loss_history,val_loss_history,train_acc_history,val_acc_history,plot_path):

### CÓDIGO ###

    plt.close(fig)


def save_results(epoch,train_loss,train_acc,val_loss,val_acc,csv_path):
    header = "epoch,train_loss,train_acc,val_loss,val_acc\n"
    data = f"### CÓDIGO ###\n"

    file_exists = os.path.exists(csv_path)

    with open(csv_path, mode='a') as file:

        if not file_exists or os.stat(csv_path).st_size == 0:
            file.write(header)
        ### CÓDIGO ###

## 6. Entrenar el modelo.

Definir todas las variables necesarias para:

* Que se guarden en un directorio "results":
  * el mejor modelo basandose en el loss.
  * el checkpoint del modelo: época, model_state_dict, optimizer_state_dict y loss
  * el plot del entrenamiento.
  * el loss y el accuracy de train y validación en un csv.

* Crea una condición de detención temprana, en la que si durante 3 épocas consecutivas no mejora 0,005 el modelo (utilizando el loss) pare el entrenamiento.
* lr = 0.001
* Número de épocas = 5
* Optimizador = Adam

In [ ]:
model = MLP_pytorch().to(device)
epochs = 5
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

path = ### CÓDIGO ###

# EarlyStopping
val_loss_history = []
patience = 3
min_delta = 0.005
early_stop_counter = 0

# Training progress Visualization
plot_namefile = "training_plot.pdf"
plot_path = path+plot_namefile
train_loss_history = []
train_acc_history = []
val_acc_history = []

# Save results as csv
csv_file = "training_results.csv"
csv_path = path+csv_file

# Model checkpoint
best_model_namefile = "best_model.pth"
best_model_path = path+best_model_namefile

checkpoint_namefile = "checkpoint.pth"
checkpoint_path = path+checkpoint_namefile
best_val_loss = float('inf')

### Bucle de entrenamiento

In [ ]:
for epoch in range(epochs):

    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:

### CÓDIGO ###

    with torch.no_grad():

### CÓDIGO ###

    print(f"Época {epoch+1}/{epochs} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    if val_loss < best_val_loss:

### CÓDIGO ###

    plot_progress(### CÓDIGO ###)

    save_results(### CÓDIGO ###)

    if epoch >= 1: # Early Stopping
      ### CÓDIGO ###

## 7. Cargar el modelo y el checkpoint

In [ ]:
model_complete = ### CÓDIGO ###

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Usa el mismo optimizador

# Cargar el checkpoint
checkpoint = torch.load(checkpoint_path)
model_complete.load_state_dict(### CÓDIGO ###)
optimizer.load_state_dict(### CÓDIGO ###)
epoch = ### CÓDIGO ###
loss = ### CÓDIGO ###

<ipython-input-16-a5648f62d148>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


## 8. Reentrenar el modelo

Reentrenar durante 20 épocas, con las mismas condiciones que en el primer entrenamiento.

In [ ]:
epochs = 20
path = ### CÓDIGO ###
model = model_complete


# EarlyStopping
val_loss_history = []
patience = 3
min_delta = 0.005
early_stop_counter = 0

# Training progress Visualization
plot_namefile = "training_plot.pdf"
plot_path = path+plot_namefile
train_loss_history = []
train_acc_history = []
val_acc_history = []

# Save results as csv
csv_file = "training_results.csv"
csv_path = path+csv_file

# Model checkpoint
best_model_namefile = "best_model.pth"
best_model_path = path+best_model_namefile

checkpoint_namefile = "checkpoint.pth"
checkpoint_path = path+checkpoint_namefile
best_val_loss = float('inf')

In [1]:
### CÓDIGO ###

## 9. Evaluar el modelo con el conjunto de Test.

Calcular el loss y el accuracy.
Guardar en listas:
* Etiquetas
* Predicciones
* Probabilidades

In [ ]:
model.eval()

# Evaluación del modelo
total_loss = 0.0
correct = 0
total = 0

all_labels = []
all_predictions = []
all_probabilities = []


with torch.no_grad():  # No calcular gradientes
    for inputs, labels in test_loader:

        ### CÓDIGO ###

        probabilities = torch.softmax(outputs, dim=1)  # Obtener probabilidades
        _, predicted = ### CÓDIGO ###

### CÓDIGO ###

print(f"Pérdida promedio en test: {avg_loss:.4f}")
print(f"Precisión en test: {accuracy:.4%}")
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)
all_probabilities = np.array(all_probabilities)

## 10. Inferencia

Muestra las predicciones para 5 muestras para cada bach, durante 6 iteraciones.

Tener en cuenta que tendréis que modificar la forma de las muestras.

In [ ]:
image_shape = (150, 150)

model.eval()
with torch.no_grad():
    for j, (inputs, labels) in enumerate(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)


        total_loss += loss.item()


        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        fig, axes = plt.subplots(1, 5, figsize=(15, 3))
        for i, (img, pred, target) in enumerate(zip(inputs, predicted, labels)):
            if i >= 5:
                break

### CÓDIGO ###

        if j == 6:
          break

## 11. Visualizaciones

### Reporte de clasificación de Sklearn

In [4]:
print("Reporte de Clasificación:\n")
### CÓDIGO ###

Reporte de Clasificación:



### Matriz de confusión

In [3]:
### CÓDIGO ###

### Curva Roc global

In [ ]:
classes = np.unique(all_labels)
y_true_bin = label_binarize(all_labels, classes=classes)

fpr = {}
tpr = {}
roc_auc = {}

In [5]:
### CÓDIGO ###

### Curva Roc por clases

In [2]:
### CÓDIGO ###